<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 2<br/>*POS taggers* pour le français dans spaCy et NLTK

**Objectif**

Comparer l'étiqueteur morphosyntaxique français prêt-à-l'emploi de spaCy avec deux étiqueteurs entraînés, l'un dans spaCy et l'autre dans NLTK.

## 1. Installation et test de spaCy

La boîte à outils spaCy est une librairie Python *open source* pour le TAL, dédiée à un usage en production. Les documents suivants vous seront utiles :
* comment [installer](https://spacy.io/usage) spaCy
* comment [télécharger un modèle](https://spacy.io/usage/models) pour une langue donnée (on appelle ces modèles des *trained pipelines* car ils enchaînent plusieurs traitements)
* comment faire les [premiers pas](https://spacy.io/usage/spacy-101) dans l'utilisation de spaCy

Veuillez installer spaCy, puis la *pipeline* pour le français appelée `fr_core_news_sm`.  Si vous utilisez *conda*, installez spaCy dans l'environnement du cours TAL.

In [5]:
import spacy
nlp = spacy.load("fr_core_news_sm") # charge la pipeline
import tqdm # permet l'affichage d'une barre de progression

**1a.** Une pipeline effectue un ensemble de traitements d'un texte en lui ajoutant des annotations.  Les traitements effectués par la pipeline `fr_core_news_sm` sont [documentés ici](https://spacy.io/models/fr#fr_core_news_sm).  La liste des traitements d'une pipeline figure dans son attribut `.pipe_names`.  On peut activer ou désactiver un traitement T avec, respectivement, les méthodes `.disable_pipe(T)` et `.enable_pipe(T)` appliquées à la pipeline.

* Veuillez afficher les traitements disponibles dans la pipeline `fr_core_news_sm` chargée ci-dessus sous le nom de `nlp` .
* Veuillez désactiver tous les traitements sauf `tok2vec` et `morphologizer` (on fait cela pour accélerer le traitement).
* Vérifiez que la désactivation a bien fonctionné en affichant les traitements activés.

In [6]:
print(nlp.pipe_names)
nlp.disable_pipe("parser")
nlp.disable_pipe("attribute_ruler")
nlp.disable_pipe("lemmatizer")
nlp.disable_pipe("ner")
print(nlp.pipe_names)

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
['tok2vec', 'morphologizer']


In [7]:
from spacy.lang.fr.examples import sentences

**1b.** L'objet `sentences` chargé ci-dessus contient une liste de phrases en français. 

* Veuillez afficher les deux premières phrases de `sentences`.
* Veuillez analyser chacune de ces deux phrases avec la pipeline `nlp` puis afficher chaque token et son POS tag.
    * indication : aidez-vous de la [documentation](https://spacy.io/models/fr#fr_core_news_sm) de `fr_core_news_sm`
    * consigne d'affichage : indiquer le tag entre crochets après chaque token, comme ceci : Les \[DET\] robots \[NOUN\] ...
    * note : la documentation détaillée du POS tagging dans spaCy est [disponible ici](https://spacy.io/usage/linguistic-features)
* Veuillez commenter la tokenisation et les POS tags observés : vous semblent-ils corrects pour les deux phrases ?

In [8]:
print(sentences[:2])
print()

for sentence in sentences[:2]:
    doc = nlp(sentence)
    tagged_sentence = " ".join([f"{token.text} [{token.pos_}]" for token in doc])
    print(tagged_sentence)

['Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars', "Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs"]

Apple [NOUN] cherche [NOUN] à [ADP] acheter [VERB] une [DET] start [NOUN] - [NOUN] up [ADJ] anglaise [NOUN] pour [ADP] 1 [NUM] milliard [NOUN] de [ADP] dollars [NOUN]
Les [DET] voitures [NOUN] autonomes [ADJ] déplacent [ADV] la [DET] responsabilité [NOUN] de [ADP] l' [DET] assurance [NOUN] vers [ADP] les [DET] constructeurs [NOUN]


  > Pour la première phrase :
    > - Le verbe `cherche` n'est pas reconnu en tant que tel.
    > - La ponctuation `-` dans `start-up` est considéré comme un nom à part entier.

  > Pour la seconde phrase :
    > - Le verbe `déplacent` n'est pas reconnu en tant que tel.

## 2. Prise en main des données

Les données sont fournies dans un format tabulaire dans l'archive `UD_French-GSD.zip` sur Cyberlearn.  Elles sont basées sur les données fournies par le projet [Universal Dependencies](https://github.com/UniversalDependencies/UD_French-GSD).  Leur format, appelé CoNLL-U, est [documenté ici](https://universaldependencies.org/format.html).  Veuillez placer les trois fichiers contenus dans l'archive dans un sous-dossier de ce notebook nommé `spacy_data`.

Les trois fichiers contiennent des phrases en français annotées avec les POS tags :
* le fichier `fr-ud-train.conllu` est destiné à l'entraînement
* le fichier `fr-ud-dev.conllu` est destiné aux tests préliminaires et aux réglages des paramètres
* le fichier `fr-ud-test.conllu` est destiné à l'évaluation finale.

**2a.** En inspectant les fichiers avec un éditeur texte, veuillez déterminer dans quelle colonne se trouvent les *tokens* des textes originaux, et dans quelle colonne se trouvent leurs étiquettes morpho-syntaxiques correctes (*POS tags*).  Que contient la troisième colonne ?

> - Première colonne : Index du token/ligne, commençant à 1
> - Deuxième colonne : tokens des textes originaux
> - Troisième colonne : tokens lemmatizés
> - Quatrième colonne : tags POS
> - ...

**2b.** Veuillez convertir les trois fichiers de données en des fichiers binaires utilisables par spaCy, en utilisant la [commande 'convert' fournie par spaCy](https://spacy.io/api/cli#convert).  La commande est donnée ci-dessous, le premier dossier `./input_data` contient les 3 fichiers `.conllu` et le dossier `./spacy-data` contiendra les 3 résultats.

* Veuillez exécuter la commande de conversion.
* Combien de phrases environ (à 10 phrases près) contient chaque fichier (*train*, *dev*, *test*) ?  Observez la commande et son résultat pour répondre.

In [9]:
!python -m spacy convert ./input_data ./spacy_data --converter conllu  --n-sents 10 --lang fr

ℹ Grouping every 10 sentences into a document.
✔ Generated output file (148 documents): spacy_data/fr-ud-dev.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (42 documents): spacy_data/fr-ud-test.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (1456 documents):
spacy_data/fr-ud-train.spacy


> - Dev : ~1'480
> - Test : ~420
> - Train : ~14'560

**2c**. Les données des fichiers convertis peuvent être chargées dans un objet de type `DocBin`.  Dans notre cas, un tel objet contient un ensemble de documents, chacun contenant 10 phrases.  Chaque document est un objet de type `Doc`.  Le code donné ci-dessous vous permet de charger les données de test et vous montre comment les afficher.

* Veuillez stocker la première phrase des données de test dans une variable nommée `premiere_phrase_test`.
* Veuillez afficher cette phrase, ainsi que son type dans spaCy.

In [10]:
from spacy.tokens import DocBin
from spacy.tokens import Doc
test_data = DocBin().from_disk("./spacy_data/fr-ud-test.spacy")
# Exemple d'utilisation (afficher toutes les phrases)
#for doc in test_data.get_docs(nlp.vocab):
#    for sent in doc.sents:
#        print(sent)

In [11]:
premiere_phrase_test = list(list(test_data.get_docs(nlp.vocab))[0].sents)[0]
print(premiere_phrase_test)
print()
print(type(premiere_phrase_test))

Je sens qu'entre ça et les films de médecins et scientifiques fous que nous avons déjà vus, nous pourrions emprunter un autre chemin pour l'origine.

<class 'spacy.tokens.span.Span'>


## 3. Évaluation du POS tagger français de la pipeline `fr_core_news_sm`

**3a.** Veuillez effectuer le *POS tagging* avec spaCy de la `premiere_phrase_test` et afficher les résultats dans le format demandé au (1b).  Indication : convertissez la `premiere_phrase_test` dans un objet de type `Doc` en lui appliquant la méthode `.as_doc()`.  Cet objet peut être ensuite traité par la pipeline `nlp`.

In [24]:
doc = premiere_phrase_test.as_doc()
tagged_sentence = " ".join([f"{token.text} [{token.pos_}]" for token in doc])
print(tagged_sentence)

Je [PRON] sens [VERB] qu' [SCONJ] entre [ADP] ça [PRON] et [CCONJ] les [DET] films [NOUN] de [ADP] médecins [NOUN] et [CCONJ] scientifiques [NOUN] fous [ADJ] que [PRON] nous [PRON] avons [AUX] déjà [ADV] vus [VERB] , [PUNCT] nous [PRON] pourrions [VERB] emprunter [VERB] un [DET] autre [ADJ] chemin [NOUN] pour [ADP] l' [DET] origine [NOUN] . [PUNCT]


**3b.** Veuillez afficher les tags corrects de `premiere_phrase_test`, puis comparez-les visuellement les tags trouvés automatiquement au (3a).  Quelles différences trouvez-vous ?

In [30]:
comparison = []

for token in doc:
    element = [token.text, token.pos_, token.tag_]
    comparison.append(element)

for elem in comparison:
    if elem[1] != elem[2]:
        print(f"{elem[0]} : {elem[1]} vs. {elem[2]}")

> Aucune différence n'est observée entre le fichier de test initial et celui-ci

In [31]:
from spacy.scorer import Scorer
from spacy.training import Example

In [32]:
scorer = Scorer()

**3c.** Au lieu de compter manuellement combien de tags sont différents entre la référence et le résultat de la pipeline `nlp`, vous allez utiliser la classe `Scorer` de spaCy.  Une instance de cette classe permet de calculer les scores d'une liste d'objets de type `Exemple`, en fonction des annotations disponibles dans les objets.  Un objet de type `Exemple` contient deux objets de type `Doc`, l'un avec les annotations correctes et l'autre avec les annotations produites par une pipeline.  La [documentation de la méthode](https://spacy.io/api/scorer#score) `Scorer.score(..)` vous sera utile. 

* Veuillez calculer la justesse (*accuracy*) du *POS tagging* de `premiere_phrase_test`. 
* Veuillez justifier la valeur du score obtenu en utilisant votre réponse du (3b).

In [33]:
example = Example.from_dict(premiere_phrase_test.as_doc(), {"words": [token.text for token in premiere_phrase_test], "tags": [token.tag_ for token in premiere_phrase_test]})
scorer.score(examples=[example])["tag_acc"]

1.0

> La précision vaut 1.0 ce qui est une précision parfaite. Ceci corrobore notre observation précédente en 2b, où aucune différence n'a été observée entre les tags corrects et ceux trouvés automatiquement.

**3d.** Veuillez calculer la précision du *POS tagging* de la pipeline `nlp` sur toutes les données de test présentes dans `test_data`.  Comment se compare le score obtenu avec celui mentionné [dans la documentation](https://spacy.io/models/fr#fr_core_news_sm) du modèle `fr_core_news_sm` ?

In [34]:
examples = []
for doc in test_data.get_docs(nlp.vocab):
    for sent in doc.sents:
        example = Example(nlp(sent.as_doc()), sent.as_doc())
        examples.append(example)
print('Accuracy :', scorer.score(examples)['pos_acc'])

Accuracy : 0.9172654690618762


> Nous observons une précision de 0.91.
> La documentation du modèle `fr_core_news_sm` mentionne une évaluation d'accuracy à 0.97 pour POS_ACC.
>
> Nous observons donc une discrépance, qui peut être due à la taille du corpus d'entraînement, relativement petite.

## 4. Entraîner puis évaluer un nouveau POS tagger français dans spaCy

Le but de cette partie est d'entraîner une pipeline spaCy pour le français sur les données de `fr-ud-train.conllu`, puis de comparer le modèle obtenu avec le modèle prêt-à-l'emploi testé au point précédent.  Les [instructions d'entraînement](https://spacy.io/usage/training#quickstart) de spaCy vous montrent comment entraîner une pipeline avec un POS tagger.

**4a.** Paramétrage de l'entraînement :
* générez un fichier de départ grâce à [l'interface web](https://spacy.io/usage/training#quickstart), en indiquant que vous voulez seulement un POS tagger dans la pipeline ;
* sauvegardez le code généré par spaCy dans un fichier local `base_config.cfg` ;
* générez un fichier `config.cfg` sur votre ordinateur en exécutant la ligne de commande suivante. 

In [35]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Enfin, veuillez effectuer l'entraînement avec la ligne de commande suivante.  Faites plusieurs essais, d'abord avec un petit nombre d'époques, pour estimer le temps nécessaire et observer les messages affichés.  Puis augmentez progressivement le nombre d'époques.  Quel est le critère qui vous permet de décider que vous avez un nombre suffisant d'époques ?  Dans quel dossier se trouve le meilleur modèle ?

In [36]:
!python -m spacy train config.cfg \
  --output ./myPOStagger1 \
  --paths.train ./spacy_data/fr-ud-train.spacy \
  --paths.dev ./spacy_data/fr-ud-dev.spacy \
  --verbose

[2025-03-14 10:48:17,703] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory: myPOStagger1
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-14 10:48:18,958] [INFO] Set up nlp object from config
[2025-03-14 10:48:18,963] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-03-14 10:48:18,964] [DEBUG] Loading corpus from path: spacy_data/fr-ud-train.spacy
[2025-03-14 10:48:18,964] [INFO] Pipeline: ['tok2vec', 'tagger']
[2025-03-14 10:48:18,965] [INFO] Created vocabulary
[2025-03-14 10:48:18,965] [INFO] Finished initializing nlp object
[2025-03-14 10:48:25,233] [INFO] Initialized pipeline components: ['tok2vec', 'tagger']
✔ Initialized pipeline

============================= Training pipeline =============================
[2025-03-14 10:48:25,239] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-03-14 10:48:25,240] [DEBUG] Loading corpus from path: spacy_data/fr-ud-

> Avec 5 epochs, nous passons à une accuracy de 0.94, ce qui est le critère de la documentation. Il serait possible de l'entraîner davantage mais les progrès prennent beaucoup d'epochs supplémentaires (2 epochs pour 0.1 d'accuracy par exemple).
>
> Le modèle est sauvegardé dans le dossier `myPOStagger1`, et plus spécifiquement dans le dossier `model-best`.

**4b.**  Veuillez charger le meilleur modèle (pipeline) dans la variable `nlp2` et afficher la *POS tagging accuracy* sur le corpus de test.  Le composant de la pipeline étant un *POS tagger*, vous devrez évaluer la propriété *tag_acc*. 

In [37]:
nlp2 = spacy.load("./myPOStagger1/model-best")
for doc in test_data.get_docs(nlp2.vocab):
    for sent in doc.sents:
        example = Example(nlp2(sent.as_doc()), sent.as_doc())
        examples.append(example)
print('Accuracy :', scorer.score(examples)['pos_acc'])

Accuracy : 0.9586327345309381


## 5. Entraîner puis évaluer un POS tagger pour le français dans NLTK

Le but de cette partie est d'utiliser le POS tagger appelé *Averaged Perceptron* fourni par NLTK, en l'entraînant pour le français sur les mêmes données que ci-dessus, importées cette fois-ci avec NLTK.  Pour une introduction au POS tagging avec NLTK, voir le [Chapitre 5.1 du livre NLTK](http://www.nltk.org/book/ch05.html).

Remarques :
* pour l'anglais, des taggers pré-entraînés sont disponibles dans NLTK ;
* pour appliquer un tagger existant, on écrit `nltk.pos_tag(sentence)` où `sentence` est une liste de tokens et on obtient des paires (token, TAG) ;
* l'implémentation de *Averaged Perceptron* a été faite par [Mathew Honnibal de Explosion.AI](https://explosion.ai/blog/part-of-speech-pos-tagger-in-python), la société qui a créé spaCy.

**5a.** Veuillez charger les données d'entraînement et celles de test grâce à la classe `ConllCorpusReader` de NLTK.  [La documentation de cette classe](https://www.nltk.org/api/nltk.corpus.reader.conll.html#nltk.corpus.reader.conll.ConllCorpusReader) vous montrera comment indiquer les colonnes qui contiennent les tokens ('words') et les tags corrects ('pos').  Une fois les données chargées dans une variable, vous pouvez accéder aux phrases et aux tags avec la méthode `.tagged_sents()`.

In [ ]:
from nltk.corpus.reader.conll import ConllCorpusReader

In [39]:
data_path = './input_data'

train_data = ConllCorpusReader(data_path, 'fr-ud-train.conllu', columntypes=['ignore', 'words', 'ignore', 'pos'], encoding='utf-8', separator="\t")
test_data = ConllCorpusReader(data_path, 'fr-ud-test.conllu', columntypes=['ignore', 'words', 'ignore', 'pos'], encoding='utf-8', separator="\t")   
print(train_data.tagged_sents())

[[('Les', 'DET'), ('commotions', 'NOUN'), ('cérébrales', 'ADJ'), ('sont', 'AUX'), ('devenu', 'VERB'), ('si', 'ADV'), ('courantes', 'ADJ'), ('dans', 'ADP'), ('ce', 'DET'), ('sport', 'NOUN'), ("qu'", 'SCONJ'), ('on', 'PRON'), ('les', 'PRON'), ('considére', 'VERB'), ('presque', 'ADV'), ('comme', 'ADP'), ('la', 'DET'), ('routine', 'NOUN'), ('.', 'PUNCT')], [("L'", 'DET'), ('œuvre', 'NOUN'), ('est', 'AUX'), ('située', 'VERB'), ('dans', 'ADP'), ('la', 'DET'), ('galerie', 'NOUN'), ('des', '_'), ('de', 'ADP'), ('les', 'DET'), ('batailles', 'NOUN'), (',', 'PUNCT'), ('dans', 'ADP'), ('le', 'DET'), ('château', 'NOUN'), ('de', 'ADP'), ('Versailles', 'PROPN'), ('.', 'PUNCT')], ...]


**5b.** Pour entraîner un POS tagger du type Averaged Perceptron, vous utiliserez le sous-module `nltk.tag.perceptron` du [module NLTK contenant les taggers](http://www.nltk.org/api/nltk.tag.html).  Les fonctions d'entraînement et de test sont documentées dans ce module.  Après l'entraînement, le réseau de neurones est enregistré dans un fichier `.pickle`, qui est écrasé à chaque entraînement si vous n'en faites pas une copie.  On peut également lire un fichier `.pickle` dans un tagger.

Veuillez écrire le code pour entraîner le POS tagger sur les données d'entraînement.  Comme au (4), pensez augmenter graduellement le nombre d'époques (appelées 'itérations' dans NLTK).

Combien de temps prend l'entraînement ?  Quelle est la taille du fichier enregistré ?

In [40]:
import os
import nltk
nltk.download('averaged_perceptron_tagger') # à exécuter la première fois
from nltk.tag.perceptron import PerceptronTagger

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mehdibenzekri/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [41]:
from perceptron_patched import PerceptronTagger

In [42]:
ptagger = PerceptronTagger(load=False)

In [43]:
import time
import os

In [44]:
start_time = time.time()
ptagger.train(train_data.tagged_sents())
elapsed_time = time.time() - start_time
print(f"Training time: {elapsed_time:.2f} seconds")

Training time: 15.75 seconds


In [45]:
import pickle

In [46]:
# Sauvegarder le modèle dans un fichier .pickle
model_path = './pos_tagger_model.pickle'
with open(model_path, 'wb') as f:
    pickle.dump(ptagger, f)

# Afficher la taille du fichier enregistré
model_size = os.path.getsize(model_path) / (1024 * 1024)  # Taille en Mo
print(f"Model size: {model_size:.2f} MB")

Model size: 12.12 MB


**5c.** Veuillez évaluer le tagger sur les données de test et afficher le taux de correction.

In [47]:
from sklearn.metrics import accuracy_score, classification_report

In [48]:
# Charger le modèle enregistré
with open(model_path, 'rb') as f:
    ptagger = pickle.load(f)

# Évaluer le modèle sur les données de test
accuracy = ptagger.evaluate(test_data.tagged_sents())

# Afficher le taux de correction
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.96


/var/folders/28/2dlsrphn16g4ns2q8cfthd_r0000gn/T/ipykernel_40767/353095571.py:6: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  accuracy = ptagger.evaluate(test_data.tagged_sents())


## 6. Conclusion


| spaCy (partie 3) | spaCy (partie 4) | NLTK (partie 5) | 
|------------------|------------------|-----------------|
| tagger fourni    | tagger entraîné  | tagger entraîné |
| 0.91             | 0.94             | 0.96            |


> 1. Le tagger fourni de spaCy est celui possédant la précision la plus basse, en deça même de ce qui est indiqué dans la documentation.
> Cela peut être expliqué par la taille de notre corpus, tant entraînement que de test, relativement petit.
> 2. Sans surprise, nous arrivons à des meilleurs résultats avec entraînement que sans.
> Jusqu'à éventuellement atteindre un point d'overfitting, mais ce n'est pas notre cas ici.
> 3. Le tagger NLTK obtient des meilleurs résultats, probablement dû à sa spécialisation dans cette tâche de POS tagging, là où spaCy est plus généraliste.